импортируем нужные библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 1. Чистка данных

In [2]:
df_financial = pd.read_csv('data/financial_data.csv')

In [3]:
df_financial.head()

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович


похоже, что в данных есть пропуски. Посмотрим на количество пропусков в каждом столбце.

In [4]:
missing_values = df_financial.isnull().sum()

In [5]:
missing_percentage = (missing_values / len(df_financial)) * 100
missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage.round(2)
})

In [6]:
missing_df

,Missing Values,Percentage (%)
id,0,0.00
Причина дубля,150,33.26
Ноябрь 2022,295,65.41
Декабрь 2022,292,64.75
Январь 2023,312,69.18
Февраль 2023,306,67.85
Март 2023,283,62.75
Апрель 2023,277,61.42
Май 2023,261,57.87
Июнь 2023,261,57.87


Скорее всего пропусти в таблицы объясняют что тогда не было никаких транзакций, так что можно их заполнить нулями.

In [7]:
df_financial.fillna(0, inplace=True)

In [8]:
df_financial.isnull().sum()

id               0
Причина дубля    0
Ноябрь 2022      0
Декабрь 2022     0
Январь 2023      0
Февраль 2023     0
Март 2023        0
Апрель 2023      0
Май 2023         0
Июнь 2023        0
Июль 2023        0
Август 2023      0
Сентябрь 2023    0
Октябрь 2023     0
Ноябрь 2023      0
Декабрь 2023     0
Январь 2024      0
Февраль 2024     0
Account          0
dtype: int64

In [9]:
df_prolongations = pd.read_csv('data/prolongations.csv')

In [10]:
df_prolongations.head()

,id,month,AM
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна


In [11]:
df_prolongations.isnull().sum()

id       0
month    0
AM       0
dtype: int64

Здесь нет пропусков, давайте посмотрим на количество уникальных значений в каждом столбце.

In [12]:
df_prolongations.nunique()

id       313
month     14
AM        10
dtype: int64

In [13]:
df_financial.nunique()

id               314
Причина дубля      7
Ноябрь 2022      143
Декабрь 2022     141
Январь 2023      135
Февраль 2023     139
Март 2023        163
Апрель 2023      162
Май 2023         180
Июнь 2023        170
Июль 2023        181
Август 2023      180
Сентябрь 2023    169
Октябрь 2023     164
Ноябрь 2023      146
Декабрь 2023     138
Январь 2024       84
Февраль 2024      92
Account           10
dtype: int64

In [14]:
df_financial['Причина дубля'].value_counts(dropna=False)

Причина дубля
0                      150
первая часть оплаты    114
вторая часть оплаты     99
доп работы              38
основные работы         38
изменение ЮЛ            11
карты, банки             1
Name: count, dtype: int64

похоже, что данные готовы к дальнейшей обработке. Давайте трансформируем их так, чтобы можно было легко анализировать финансовые данные по месяцам.

In [15]:
df_financial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             451 non-null    int64 
 1   Причина дубля  451 non-null    object
 2   Ноябрь 2022    451 non-null    object
 3   Декабрь 2022   451 non-null    object
 4   Январь 2023    451 non-null    object
 5   Февраль 2023   451 non-null    object
 6   Март 2023      451 non-null    object
 7   Апрель 2023    451 non-null    object
 8   Май 2023       451 non-null    object
 9   Июнь 2023      451 non-null    object
 10  Июль 2023      451 non-null    object
 11  Август 2023    451 non-null    object
 12  Сентябрь 2023  451 non-null    object
 13  Октябрь 2023   451 non-null    object
 14  Ноябрь 2023    451 non-null    object
 15  Декабрь 2023   451 non-null    object
 16  Январь 2024    451 non-null    object
 17  Февраль 2024   451 non-null    object
 18  Account        451 non-null   

давайте создадим month mapping, чтобы преобразовать названия месяцев в числа. Это поможет нам в дальнейшем анализе и визуализации данных.

In [21]:
month_map = {
    'январь': 1, 'февраль': 2, 'март': 3, 'апрель': 4,
    'май': 5, 'июнь': 6, 'июль': 7, 'август': 8,
    'сентябрь': 9, 'октябрь': 10, 'ноябрь': 11, 'декабрь': 12
}

In [ ]:

# New Cell 3: Calculate Prolongation Coefficients

import pandas as pd
import numpy as np

print("--- df_main head for reference ---")
print(df_main.head())
print("\\n--- df_main info for reference ---")
df_main.info()

# Ensure 'shipment' is numeric, fill NaNs with 0 if any slipped through or resulted from merges
df_main['shipment'] = pd.to_numeric(df_main['shipment'], errors='coerce').fillna(0)

# Step 1: Get the shipment value for each project in its specific project_end_month_num
# This represents the "sum of shipment of the last month of implementation" for each project.
df_project_end_shipments = df_main[
    df_main['shipment_month_num'] == df_main['project_end_month_num']
].copy()
df_project_end_shipments = df_project_end_shipments[['id', 'Account_Manager', 'project_end_month_num', 'shipment']]
df_project_end_shipments.rename(columns={'shipment': 'project_end_month_shipment_value'}, inplace=True)
# Remove duplicates if any project ID has multiple entries for its end month shipment (e.g. from original financial data before aggregation)
# Taking the sum here, assuming multiple entries for the same project in its end month should be summed up.
# If it's guaranteed to be unique, this groupby won't change much.
df_project_end_shipments = df_project_end_shipments.groupby(['id', 'Account_Manager', 'project_end_month_num'], as_index=False)['project_end_month_shipment_value'].sum()


print("\\n--- df_project_end_shipments (Sample) ---")
print(df_project_end_shipments.head())
print(f"Total unique projects with end month shipment data: {df_project_end_shipments['id'].nunique()}")


# Initialize lists to store monthly results
monthly_results = []

# Months in 2023 for evaluation
evaluation_months = range(1, 13) # 1 for Jan, ..., 12 for Dec

for eval_month in evaluation_months:
    # --- Calculate Coefficient 1 (C1) ---
    # Projects that ended in the month prior to eval_month
    c1_project_end_month = (eval_month - 1 - 1 + 12) % 12 + 1 # e.g., if eval_month=1 (Jan), c1_project_end_month=12 (Dec)

    # Get all projects that ended in c1_project_end_month
    ended_for_c1 = df_project_end_shipments[df_project_end_shipments['project_end_month_num'] == c1_project_end_month]

    # Denominator D1: Sum of shipments in their project_end_month_num for projects ended in c1_project_end_month
    # Grouped by Account_Manager
    d1_by_manager = ended_for_c1.groupby('Account_Manager')['project_end_month_shipment_value'].sum()
    d1_total = ended_for_c1['project_end_month_shipment_value'].sum()

    # Numerator N1: Sum of shipments in eval_month for projects that ended in c1_project_end_month
    # These are the projects that were prolonged into the first month (eval_month)
    prolonged_for_c1_shipments = df_main[
        (df_main['id'].isin(ended_for_c1['id'])) &
        (df_main['shipment_month_num'] == eval_month) &
        (df_main['shipment'] > 0) # Ensure there was actual shipment
    ]
    n1_by_manager = prolonged_for_c1_shipments.groupby('Account_Manager')['shipment'].sum()
    n1_total = prolonged_for_c1_shipments['shipment'].sum()

    # Calculate C1
    c1_by_manager = (n1_by_manager / d1_by_manager).fillna(0).replace([np.inf, -np.inf], 0)
    c1_total = (n1_total / d1_total) if d1_total > 0 else 0

    # Store C1 results
    for manager, coeff in c1_by_manager.items():
        monthly_results.append({
            'EvaluationMonth': eval_month, 'Account_Manager': manager,
            'CoefficientType': 'C1', 'CoefficientValue': coeff,
            'Numerator': n1_by_manager.get(manager, 0), 'Denominator': d1_by_manager.get(manager, 0)
        })
    monthly_results.append({
        'EvaluationMonth': eval_month, 'Account_Manager': 'Overall',
        'CoefficientType': 'C1', 'CoefficientValue': c1_total,
        'Numerator': n1_total, 'Denominator': d1_total
    })

    # --- Calculate Coefficient 2 (C2) ---
    # Projects that ended two months prior to eval_month
    c2_project_end_month = (eval_month - 2 - 1 + 12) % 12 + 1 # e.g., if eval_month=1 (Jan), c2_project_end_month=11 (Nov)
    month_to_check_non_prolongation_c2 = (eval_month - 1 - 1 + 12) % 12 + 1 # Month after c2_project_end_month

    # Get all projects that ended in c2_project_end_month
    ended_for_c2_candidates = df_project_end_shipments[
        df_project_end_shipments['project_end_month_num'] == c2_project_end_month
    ]

    # Identify projects from ended_for_c2_candidates that were NOT prolonged in their first possible month (month_to_check_non_prolongation_c2)
    shipments_in_first_prolongation_month_c2 = df_main[
        (df_main['id'].isin(ended_for_c2_candidates['id'])) &
        (df_main['shipment_month_num'] == month_to_check_non_prolongation_c2) &
        (df_main['shipment'] > 0)
    ]
    ids_prolonged_in_first_month_c2 = shipments_in_first_prolongation_month_c2['id'].unique()

    # These are the projects relevant for C2 denominator: ended in c2_project_end_month AND NOT prolonged in month_to_check_non_prolongation_c2
    relevant_for_c2 = ended_for_c2_candidates[
        ~ended_for_c2_candidates['id'].isin(ids_prolonged_in_first_month_c2)
    ]

    # Denominator D2: Sum of shipments in their project_end_month_num for relevant_for_c2 projects
    d2_by_manager = relevant_for_c2.groupby('Account_Manager')['project_end_month_shipment_value'].sum()
    d2_total = relevant_for_c2['project_end_month_shipment_value'].sum()

    # Numerator N2: Sum of shipments in eval_month for projects in relevant_for_c2
    # These are projects ended c2_project_end_month, not prolonged in month_to_check_non_prolongation_c2, but prolonged in eval_month
    prolonged_for_c2_shipments = df_main[
        (df_main['id'].isin(relevant_for_c2['id'])) &
        (df_main['shipment_month_num'] == eval_month) &
        (df_main['shipment'] > 0) # Ensure actual shipment
    ]
    n2_by_manager = prolonged_for_c2_shipments.groupby('Account_Manager')['shipment'].sum()
    n2_total = prolonged_for_c2_shipments['shipment'].sum()

    # Calculate C2
    c2_by_manager = (n2_by_manager / d2_by_manager).fillna(0).replace([np.inf, -np.inf], 0)
    c2_total = (n2_total / d2_total) if d2_total > 0 else 0

    # Store C2 results
    for manager, coeff in c2_by_manager.items():
        monthly_results.append({
            'EvaluationMonth': eval_month, 'Account_Manager': manager,
            'CoefficientType': 'C2', 'CoefficientValue': coeff,
            'Numerator': n2_by_manager.get(manager, 0), 'Denominator': d2_by_manager.get(manager, 0)
        })
    monthly_results.append({
        'EvaluationMonth': eval_month, 'Account_Manager': 'Overall',
        'CoefficientType': 'C2', 'CoefficientValue': c2_total,
        'Numerator': n2_total, 'Denominator': d2_total
    })

# Convert results to DataFrame
df_monthly_coeffs = pd.DataFrame(monthly_results)

print("\\n--- Monthly Coefficients (Sample) ---")
print(df_monthly_coeffs.head())

print("\\n--- Monthly Coefficients for a specific Manager (Example: first manager found) ---")
if not df_monthly_coeffs.empty and 'Account_Manager' in df_monthly_coeffs.columns:
    sample_manager = df_monthly_coeffs[df_monthly_coeffs['Account_Manager'] != 'Overall']['Account_Manager'].unique()[0]
    print(df_monthly_coeffs[df_monthly_coeffs['Account_Manager'] == sample_manager])

print("\\n--- Monthly Coefficients for 'Overall' ---")
print(df_monthly_coeffs[df_monthly_coeffs['Account_Manager'] == 'Overall'])


# --- Calculate Annual Coefficients ---
# Sum of all numerators / Sum of all denominators for the year

df_annual_coeffs_calcs = df_monthly_coeffs.groupby(['Account_Manager', 'CoefficientType']).agg(
    TotalNumerator=('Numerator', 'sum'),
    TotalDenominator=('Denominator', 'sum')
).reset_index()

df_annual_coeffs_calcs['AnnualCoefficientValue'] = (
    df_annual_coeffs_calcs['TotalNumerator'] / df_annual_coeffs_calcs['TotalDenominator']
).fillna(0).replace([np.inf, -np.inf], 0)

print("\\n--- Annual Coefficients ---")
print(df_annual_coeffs_calcs)

# Pivot monthly results for better readability if needed for Excel later
df_monthly_coeffs_pivot_c1 = df_monthly_coeffs[df_monthly_coeffs['CoefficientType']=='C1'].pivot_table(
    index='EvaluationMonth', columns='Account_Manager', values='CoefficientValue'
).fillna(0)
df_monthly_coeffs_pivot_c2 = df_monthly_coeffs[df_monthly_coeffs['CoefficientType']=='C2'].pivot_table(
    index='EvaluationMonth', columns='Account_Manager', values='CoefficientValue'
).fillna(0)

print("\\n--- Monthly C1 Coefficients (Pivoted) ---")
print(df_monthly_coeffs_pivot_c1)
print("\\n--- Monthly C2 Coefficients (Pivoted) ---")
print(df_monthly_coeffs_pivot_c2)

# For annual, a similar pivot or just use df_annual_coeffs_calcs
df_annual_coeffs_pivot = df_annual_coeffs_calcs.pivot_table(
    index='Account_Manager', columns='CoefficientType', values='AnnualCoefficientValue'
).fillna(0)

print("\\n--- Annual Coefficients (Pivoted) ---")
print(df_annual_coeffs_pivot)

# These DataFrames (df_monthly_coeffs_pivot_c1, df_monthly_coeffs_pivot_c2, df_annual_coeffs_pivot, and df_annual_coeffs_calcs for detailed N/D)
# are now ready for export to Excel or further visualization.
